# Chargement des id et des probas

In [1]:
import pandas as pd
df = pd.read_csv("data/training_solutions_rev1.csv", usecols=[0,1,2,3])
df

,GalaxyID,Class1.1,Class1.2,Class1.3
0,100008,0.383147,0.616853,0.000000
1,100023,0.327001,0.663777,0.009222
2,100053,0.765717,0.177352,0.056931
3,100078,0.693377,0.238564,0.068059
4,100090,0.933839,0.000000,0.066161
...,...,...,...,...
61573,999948,0.510379,0.489621,0.000000
61574,999950,0.901216,0.098784,0.000000
61575,999958,0.202841,0.777376,0.019783
61576,999964,0.091000,0.909000,0.000000


# Suppression de la classe 3 qui n'a que 59 images...

In [2]:
df = df.drop(df[(df['Class1.3'] > df['Class1.2']) & (df['Class1.3'] > df['Class1.1'])].index)
df = df.drop(['Class1.3'], axis=1)
df.shape

(61519, 3)

# Échantillons test et d'entraînement

In [3]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=.35)
df_train.shape, df_test.shape

((39987, 3), (21532, 3))

# Chargement des images, crop, resize et normalisation

In [4]:
import cv2
from tqdm import tqdm
import numpy as np

shape_in =  (424,424)
crop =      (256,256)
shape_out = (64,64)
    
def get_crop_resize_images(df, shape_out=shape_out, crop=crop):
    x_start = (shape_in[0] - crop[0])//2
    y_start = (shape_in[1] - crop[1])//2
    
    x_end = x_start + crop[0]
    y_end = y_start + crop[1]
   
    val = df.values
    ids = val[:,0].astype(int).astype(str)
    y = val[:,1:]
    X = []
    
    for i in tqdm(ids):
        x = cv2.imread(f'data/images_training_rev1/{i}.jpg')
        x = x[x_start:x_end, y_start:y_end]
        x = cv2.resize(x, shape_out)
        x = x/255.
        X.append(x)        
    X = np.array(X)
    return X, y

In [5]:
#X_train, y_train = get_crop_resize_images(df_train)

In [6]:
#X_test, y_test = get_crop_resize_images(df_test)

In [7]:
# plus compliqué mais plus rapide : version en parallélisant les calculs
from multiprocessing import Pool

with Pool(5) as p:
    res_train = p.map(get_crop_resize_images, np.array_split(df_train, 10))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3998/3998 [00:12<00:00, 315.93it/s]


In [8]:
with Pool(5) as p:
    res_test = p.map(get_crop_resize_images, np.array_split(df_test, 10))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2153/2153 [00:07<00:00, 306.56it/s]


In [9]:
X_train = np.vstack([res_train[k][0] for k in range(len(res_train))])

In [10]:
X_test = np.vstack([res_test[k][0] for k in range(len(res_test))])

In [11]:
y_train = np.vstack([res_train[k][1] for k in range(len(res_train))])

In [12]:
y_test = np.vstack([res_test[k][1] for k in range(len(res_test))])

In [13]:
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

X_train (39987, 64, 64, 3)
y_train (39987, 2)
X_test (21532, 64, 64, 3)
y_test (21532, 2)


# Sauvegarde des array

In [15]:
np.save('arrays/X_train', X_train)
np.save('arrays/y_train', y_train)
np.save('arrays/X_test', X_test)
np.save('arrays/y_test', y_test)